Para enriquecer nuestra base de datos, extraeremos el valor de mercado de cada jugardor de La Liga en la temporada 2019-2020.
Obtendremos esta información de www.transfermarkt.com, portal de referencia en el cálculo de esta métrica.

Sin embargo, nos encontramos con un problema:
    - No podemos acceder a una única url que contenga el valor de mercado para todos los jugadores de La Liga.
    
Lo que nos encontramos es una página en la que se muestra el valor total para cada equipo (junto con otros datos), y a partir de ahí podemos ir accediendo uno a uno a los equipos para ver el valor de cada uno de sus jugadores (https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2019).

Sí que hay una url que nos muestra el valor para todos los jugadores de La Liga, pero esta información hace referencia a la temporada actual y no a la temporada objeto de estudio en nuestro proyecto.

Esto implica que no podamos hacer un único web scrapping a partir del cual obtengamos todos los datos que nos interesan. Tendremos que hacer un scrap para cada equipo de la liga.

Selenium podría resolvernos de manera más eficiente el problema (nos permite navegar por la página web que queremos scrapear), pero en este proyecto no utilizaremos esta herramienta.

Por otro lado, no tenemos accesos a la API de Transfermarkt en la que se presupone que estarían disponibles todos estos datos. Sin embargo, existe una API generada por terceros que sí nos facilita mucho esta tarea: https://apify.com/petr_cermak/transfermarkt.

Utilizaremos esta API para acceder a la información de las diferentes páginas de www.transfermarkt.com y, como veremos más adelante, el manejo de la API es muy sencillo. Realmente se gestiona a través de una interfaz gráfica en la que incluimos la url que queremos scrapear y nos devuelve otra url (o archivo) con la información solicitada. A partir de ahí, haremos las requests y manejaremos sus respectivos json para obtener el resultado deseado.


In [3]:
import requests as req
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

# Web Scrapping (transfermarkt).

Comenzamos por el primer equipo de La Liga según orden alfabético: Athletic Club (según la nomenclatura utilizada en Teams Stats).

Tendremos que seguir el mismo proceso para cada equipo de La Liga y sus respectivas URLs proporcionadas por la API. Podemos generar un data frame para cada uno de ellos y concatenarlos todos en un único DataFrame

In [13]:
url = 'https://api.apify.com/v2/datasets/wAt2WvPDhZ51fRFq3/items?clean=true&format=json'

In [14]:
bilbao =req.get(url)

In [35]:
bilbao.json() # Mostramos el json esta primera vez para entender mejor el proceso, pero se omitirá en el resto de los equipos.

[{'id': '621',
  'url': 'https://www.transfermarkt.com/athletic-bilbao/startseite/verein/621/saison_id/2019',
  'type': 'club',
  'squadSize': '25',
  'averageAge': '26.1',
  'foreigners': ['0', ' %'],
  'nationalTeamPlayers': '2',
  'stadium': ['San Mamés', '53.289 Seats'],
  'currentTransferRecord': '+-0',
  'players': [{'number': '1',
    'player': ['Unai Simón', 'Goalkeeper'],
    'marketValue': '€16.00m',
    'url': 'https://www.transfermarkt.com/unai-simon/profil/spieler/262396',
    'id': '262396',
    'nat.': 'Spain'},
   {'number': '13',
    'player': ['Iago Herrerín', 'Goalkeeper'],
    'marketValue': '€2.80m',
    'url': 'https://www.transfermarkt.com/iago-herrerin/profil/spieler/71570',
    'id': '71570',
    'nat.': 'Spain'},
   {'number': '31',
    'player': ['Jokin Ezkieta', 'Goalkeeper'],
    'marketValue': '€300Th.',
    'url': 'https://www.transfermarkt.com/jokin-ezkieta/profil/spieler/379548',
    'id': '379548',
    'nat.': 'Spain'},
   {'number': '35',
    'player'

In [38]:
# Observamos que el contenido que nos interesa está vinculado a la key 'players', así que estraemos este valor.

bilbao_players = []

for e in bilbao.json():
    bilbao_players.append(e['players'])

In [39]:
bilbao_players 

# Es una lista dentro de otra y esto da problemas a la hora de generar el DataFrame (nos los incluiría todo en una única fila).

[[{'number': '1',
   'player': ['Unai Simón', 'Goalkeeper'],
   'marketValue': '€16.00m',
   'url': 'https://www.transfermarkt.com/unai-simon/profil/spieler/262396',
   'id': '262396',
   'nat.': 'Spain'},
  {'number': '13',
   'player': ['Iago Herrerín', 'Goalkeeper'],
   'marketValue': '€2.80m',
   'url': 'https://www.transfermarkt.com/iago-herrerin/profil/spieler/71570',
   'id': '71570',
   'nat.': 'Spain'},
  {'number': '31',
   'player': ['Jokin Ezkieta', 'Goalkeeper'],
   'marketValue': '€300Th.',
   'url': 'https://www.transfermarkt.com/jokin-ezkieta/profil/spieler/379548',
   'id': '379548',
   'nat.': 'Spain'},
  {'number': '35',
   'player': ['Ander Iru', 'Goalkeeper'],
   'marketValue': '€50Th.',
   'url': 'https://www.transfermarkt.com/ander-iru/profil/spieler/480958',
   'id': '480958',
   'nat.': 'Spain'},
  {'number': '4',
   'player': ['Iñigo Martínez', 'Centre-Back'],
   'marketValue': '€25.50m',
   'url': 'https://www.transfermarkt.com/inigo-martinez/profil/spieler/1

In [31]:
# Aplanamos la lista y obtenemos el resultado deseado.

bilbao_def = []

for lista in bilbao_players:
    for e in lista:
        bilbao_def.append(e)

In [33]:
# Convertimos el json en DataFrame.

bilbao_df = pd.DataFrame(bilbao_def)

In [37]:
bilbao_df.head()

,number,player,marketValue,url,id,nat.
0,1,"[Unai Simón, Goalkeeper]",€16.00m,https://www.transfermarkt.com/unai-simon/profi...,262396,Spain
1,13,"[Iago Herrerín, Goalkeeper]",€2.80m,https://www.transfermarkt.com/iago-herrerin/pr...,71570,Spain
2,31,"[Jokin Ezkieta, Goalkeeper]",€300Th.,https://www.transfermarkt.com/jokin-ezkieta/pr...,379548,Spain
3,35,"[Ander Iru, Goalkeeper]",€50Th.,https://www.transfermarkt.com/ander-iru/profil...,480958,Spain
4,4,"[Iñigo Martínez, Centre-Back]",€25.50m,https://www.transfermarkt.com/inigo-martinez/p...,158863,Spain


Este es el proceso que debemos seguir para cada equipo de La Liga (al tener los datos de cada equipo en urls diferentes, tendríamos que repetir el proceso para cada url).

Esto quiere decir que, si lo hiciésemos una a una, tendríamos que repetir el proceso otras 19 veces.

Pero... ¿No podemos hacer un loop para "automatizar" este proceso?

Claro que podemos y, de hecho, es así como lo haremos.

In [42]:
# En primer lugar, necesitamos obtener todas las urls de la API y generar una lista sobre la que iterar a la hora de ejecutar las requests.
# Este proceso puede resultar tedioso, pero es necesario para mejorar la eficiencia de nuestro código a través del loop.

all_urls = ['https://api.apify.com/v2/datasets/wAt2WvPDhZ51fRFq3/items?clean=true&format=json',
           'https://api.apify.com/v2/datasets/vp946DfSTHcf8bpiG/items?clean=true&format=json',
           'https://api.apify.com/v2/datasets/AN399tXVA9Ub3WaRZ/items?clean=true&format=json',
           'https://api.apify.com/v2/datasets/mtrwr19S5KsTAn9e9/items?clean=true&format=json',
           'https://api.apify.com/v2/datasets/NxGqnrEJe7uk1MB5k/items?clean=true&format=json',
           'https://api.apify.com/v2/datasets/VRFLvKqfkgIPyImbx/items?clean=true&format=json',
           ]

In [44]:
all_players = []

for e in all_urls:
    team = req.get(e)
    team_players = []
    for i in team.json():
        team_players.append(i['players'])
        team_def = []
        for x in team_players:
            for y in x:
                team_def.append(y)
            all_players.append(team_def)
        

In [54]:
df_list = []

for lista in all_players:
    for e in lista:
        df_list.append(e)

In [55]:
df_list

[{'number': '1',
  'player': ['Unai Simón', 'Goalkeeper'],
  'marketValue': '€16.00m',
  'url': 'https://www.transfermarkt.com/unai-simon/profil/spieler/262396',
  'id': '262396',
  'nat.': 'Spain'},
 {'number': '13',
  'player': ['Iago Herrerín', 'Goalkeeper'],
  'marketValue': '€2.80m',
  'url': 'https://www.transfermarkt.com/iago-herrerin/profil/spieler/71570',
  'id': '71570',
  'nat.': 'Spain'},
 {'number': '31',
  'player': ['Jokin Ezkieta', 'Goalkeeper'],
  'marketValue': '€300Th.',
  'url': 'https://www.transfermarkt.com/jokin-ezkieta/profil/spieler/379548',
  'id': '379548',
  'nat.': 'Spain'},
 {'number': '35',
  'player': ['Ander Iru', 'Goalkeeper'],
  'marketValue': '€50Th.',
  'url': 'https://www.transfermarkt.com/ander-iru/profil/spieler/480958',
  'id': '480958',
  'nat.': 'Spain'},
 {'number': '4',
  'player': ['Iñigo Martínez', 'Centre-Back'],
  'marketValue': '€25.50m',
  'url': 'https://www.transfermarkt.com/inigo-martinez/profil/spieler/158863',
  'id': '158863',
  

In [56]:
df = pd.DataFrame(df_list)

In [57]:
df

,number,player,marketValue,url,id,nat.
0,1,"[Unai Simón, Goalkeeper]",€16.00m,https://www.transfermarkt.com/unai-simon/profi...,262396,Spain
1,13,"[Iago Herrerín, Goalkeeper]",€2.80m,https://www.transfermarkt.com/iago-herrerin/pr...,71570,Spain
2,31,"[Jokin Ezkieta, Goalkeeper]",€300Th.,https://www.transfermarkt.com/jokin-ezkieta/pr...,379548,Spain
3,35,"[Ander Iru, Goalkeeper]",€50Th.,https://www.transfermarkt.com/ander-iru/profil...,480958,Spain
4,4,"[Iñigo Martínez, Centre-Back]",€25.50m,https://www.transfermarkt.com/inigo-martinez/p...,158863,Spain
...,...,...,...,...,...,...
2175,9,"[Álvaro Morata, Centre-Forward]",€36.00m,https://www.transfermarkt.com/alvaro-morata/pr...,128223,Spain
2176,19,"[Diego Costa, Centre-Forward]",€14.50m,https://www.transfermarkt.com/diego-costa/prof...,44779,"[Spain, Brazil]"
2177,34,"[Sergio Camello, Centre-Forward]",€2.30m,https://www.transfermarkt.com/sergio-camello/p...,505674,Spain
2178,17,"[Ivan Saponjic, Centre-Forward]",€1.20m,https://www.transfermarkt.com/ivan-saponjic/pr...,274347,Serbia


In [1]:
url = 'https://api.apify.com/v2/datasets/9W3MXNcnRa4BCa3M1/items?clean=true&format=json'

In [10]:
extra_teams =req.get(url)

In [11]:
extra_teams

<Response [200]>

In [12]:
extra_teams.json()

[{'url': 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2019',
  'type': 'competition',
  'leagueLevel': ['First Tier\xa0-', 'Spain'],
  'numberOfTeams': '20 teams',
  'players': '511',
  'foreigners': ['222\xa0Players', '43.4%'],
  'avgMarketValue': '€9.66m',
  'uefaCoefficient': ['2. Pos.', '89.855 Points'],
  'recordHoldingChampions': ['Real Madrid', '34\xa0time(s)'],
  'avgAge': '27.8',
  'reigningChampion': 'Atlético de Madrid',
  'clubs': [{'name': 'FC Barcelona',
    'avgAge': '24.4',
    'foreigners': '25',
    'avgMarketValue': '€21.65m',
    'totalMarketValue': '€930.93m',
    'url': 'https://www.transfermarkt.com/fc-barcelona/startseite/verein/131/saison_id/2019',
    'id': '2019'},
   {'name': 'Real Madrid',
    'avgAge': '25.6',
    'foreigners': '20',
    'avgMarketValue': '€27.69m',
    'totalMarketValue': '€913.75m',
    'url': 'https://www.transfermarkt.com/real-madrid/startseite/verein/418',
    'id': '418'},
   {'name': 'Atlético de M